In [5]:
import tweepy
import time
import json
import io
from random import uniform
import time
from IPython.display import display, clear_output
#https://developer.twitter.com/en/docs/basics/rate-limiting.html
#https://developer.twitter.com/en/apps
#author: omar edgardo lugo sanchez lugo.oe@gmail.com
    

In [6]:
class MyStreamListener(tweepy.StreamListener):
    def write_to_file_f(self):
        try:
            display("Writing tweets to file "+self.write_to_file+".json")
            display("Tweets: "+str(len(self.tweet_data)))
            with io.open(self.write_to_file, 'a', encoding='utf8') as json_file:
                for tweet in self.tweet_data:
                    json.dump(tweet, json_file, ensure_ascii=False)
            json_file.close()
            display("Tweets writen!")
            time.sleep(3)  
            self.tweet_data = []
        except Exception as e:
            print("ERROR: {}".format(e))            
    def on_status(self, status): 
        #code to run each time the stream receives a status
        print(status)
        print(status.text)
    def on_direct_message(self, status): 
        #code to run each time the stream receives a direct message
        print(status)
        print(status.text)
    def __init__(self, start_time, time_limit=900,file="raw_tweets"):
        self.time = start_time
        self.limit = time_limit
        self.tweet_data = []
        self.write_to_file = file
        self.total_tweets = 0
    def on_data(self, data):
        clear_output(wait=True)
        display('Time: '+str(round((time.time() - self.time)/60,3))+' Min. |  Remaining time: '+str(round((self.limit-(time.time() - self.time))/60,3))+" Min.")
        display('Total tweets: '+str(self.total_tweets))
        if ((time.time() - self.time) < self.limit):
            all_data = json.loads(data)
            if 'text' in all_data:
                self.total_tweets = self.total_tweets + 1
                self.tweet_data.append(data)
                if (len(self.tweet_data)>0 and len(self.tweet_data)%2==0):
                    self.write_to_file_f()
            else:
                print("Not a tweet!")
                print(data)
            time.sleep(1)    
            return True
        else:
            display("Time up! closing stream...")
            if (len(self.tweet_data)>0):
                display("Writing Remaining tweets...")
                self.write_to_file_f()
            time.sleep(1)
        return False
    def on_error(self, status_code):
        #code to run each time an error is received
        if status_code == 420:
            print("Returned when an app is being rate limited for making too many requests.")
            return False
        else:
            return True
       

In [ ]:
Sleep_time_seconds = 60
Extracting_tweets_time_in_seconds = 120
while(True):
    clear_output(wait=True)
    display("Begin Stream Listener")
    #Consumer API keys
    consumer_key = 'goFfzbLgPEucyElAUWCzxAxGb' #(API key)
    consumer_secret = 'AhIMisrlKxnfZYFqxU64RR1iBimdJKkhWMEM1oAMZKCA7VylkA' #(API secret key)

    #Access token & access token secret
    access_token = '1155691399817433088-X4FDoCudfgASnT5f20WbgXGmY9qtQc' #(Access token)
    access_token_secret = 'pQeD7FqP8O7S0djTXWEVAkjBnyYEZhdm3qBeDYziduMbG' #(Access token secret)

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)

    myStreamListener = MyStreamListener(start_time=time.time(),time_limit=Extracting_tweets_time_in_seconds,file = "raw_tweets")
    myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)
    myStream.filter(track=['موت'],languages=['ur'])
    display("Sleeping for: "+str(round(Sleep_time_seconds/60,3))+" minutes in order to avoid limit rate...")
    time.sleep(Sleep_time_seconds)